In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_stata("prop99_example.dta")

In [ ]:
# indep, state, year, treated

In [4]:
df['ty'] = np.where(df.treated==1, df.year, np.nan)
df['treated_'] = df.groupby('state').treated.transform('max')
df['tyear'] = df.groupby('state').ty.transform('min')

In [5]:
df.sort_values(['year', 'treated_', 'state'], inplace=True)

In [7]:
rnames = df[df.treated==0].state.unique()

In [10]:
df['n'] = np.arange(1, df.shape[0] + 1)

In [13]:
mint = df.year.min()